# 连表查询-定义一对多/多对一表


## 测试准备


### 定义model

In [1]:
from typing import Optional

from sqlmodel import Field, SQLModel, create_engine


class Team(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(index=True)
    headquarters: str


class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(index=True)
    secret_name: str
    age: Optional[int] = Field(default=None, index=True)

    team_id: Optional[int] = Field(default=None, foreign_key="team.id")

### 创建表

In [7]:
sqlite_file_name = "join_database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"

engine = create_engine(sqlite_url, echo=False)

SQLModel.metadata.create_all(engine)
from sqlmodel import Session
from sqlmodel import select


### 添加测试数据

In [2]:

with Session(engine) as session:
    team_preventers = Team(name="Preventers", headquarters="Sharp Tower")
    team_z_force = Team(name="Z-Force", headquarters="Sister Margaret's Bar")
    session.add(team_preventers)
    session.add(team_z_force)
    session.commit()

    hero_deadpond = Hero(
        name="Deadpond", secret_name="Dive Wilson", team_id=team_z_force.id
    )
    hero_rusty_man = Hero(
        name="Rusty-Man",
        secret_name="Tommy Sharp",
        age=48,
        team_id=team_preventers.id,
    )
    hero_spider_boy = Hero(name="Spider-Boy", secret_name="Pedro Parqueador")

    session.add(hero_deadpond)
    session.add(hero_rusty_man)
    session.add(hero_spider_boy)
    session.commit()
    session.refresh(hero_deadpond)
    session.refresh(hero_rusty_man)
    session.refresh(hero_spider_boy)

    print("Created hero:", hero_deadpond)
    print("Created hero:", hero_rusty_man)
    print("Created hero:", hero_spider_boy)

Created hero: team_id=2 name='Deadpond' secret_name='Dive Wilson' age=None id=1
Created hero: team_id=1 name='Rusty-Man' secret_name='Tommy Sharp' age=48 id=2
Created hero: team_id=None name='Spider-Boy' secret_name='Pedro Parqueador' age=None id=3


## 连表方法

使用where和使用join的sql是不同的

### 方式一 where连表

In [6]:
with Session(engine) as session:
    statement = select(Hero, Team).where(Hero.team_id == Team.id)
    results=session.exec(statement).all()
    for hero, team in results:
        print("Hero:", hero, "  Team:", team)

2023-12-19 15:46:04,686 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-19 15:46:04,692 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id, team.id AS id_1, team.name AS name_1, team.headquarters 
FROM hero, team 
WHERE hero.team_id = team.id
2023-12-19 15:46:04,693 INFO sqlalchemy.engine.Engine [generated in 0.00105s] ()
Hero: id=1 age=None name='Deadpond' secret_name='Dive Wilson' team_id=2   Team: name='Z-Force' headquarters="Sister Margaret's Bar" id=2
Hero: id=2 age=48 name='Rusty-Man' secret_name='Tommy Sharp' team_id=1   Team: name='Preventers' headquarters='Sharp Tower' id=1
2023-12-19 15:46:04,695 INFO sqlalchemy.engine.Engine ROLLBACK


### 方式二 join连表

In [8]:
with Session(engine) as session:
    statement = select(Hero, Team).join(Team)
    results = session.exec(statement)
    for hero, team in results:
        print("Hero:", hero, "  Team:", team)

2023-12-19 15:47:40,645 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-19 15:47:40,647 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id, team.id AS id_1, team.name AS name_1, team.headquarters 
FROM hero JOIN team ON team.id = hero.team_id
2023-12-19 15:47:40,647 INFO sqlalchemy.engine.Engine [cached since 48.42s ago] ()
Hero: id=1 age=None name='Deadpond' secret_name='Dive Wilson' team_id=2   Team: name='Z-Force' headquarters="Sister Margaret's Bar" id=2
Hero: id=2 age=48 name='Rusty-Man' secret_name='Tommy Sharp' team_id=1   Team: name='Preventers' headquarters='Sharp Tower' id=1
2023-12-19 15:47:40,648 INFO sqlalchemy.engine.Engine ROLLBACK


## 连接方向

### 左 

In [10]:
with Session(engine) as session:
    statement = select(Hero, Team).join(Team,)
    results = session.exec(statement)
    for hero, team in results:
        print("Hero:", hero, "Team:", team)

2023-12-19 16:11:05,814 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-19 16:11:05,815 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id, team.id AS id_1, team.name AS name_1, team.headquarters 
FROM hero JOIN team ON team.id = hero.team_id
2023-12-19 16:11:05,816 INFO sqlalchemy.engine.Engine [cached since 1454s ago] ()
Hero: id=1 age=None name='Deadpond' secret_name='Dive Wilson' team_id=2 Team: name='Z-Force' headquarters="Sister Margaret's Bar" id=2
Hero: id=2 age=48 name='Rusty-Man' secret_name='Tommy Sharp' team_id=1 Team: name='Preventers' headquarters='Sharp Tower' id=1
2023-12-19 16:11:05,818 INFO sqlalchemy.engine.Engine ROLLBACK


### 外

In [9]:
with Session(engine) as session:
    statement = select(Hero, Team).join(Team, isouter=True)
    results = session.exec(statement)
    for hero, team in results:
        print("Hero:", hero, "Team:", team)

2023-12-19 16:09:33,383 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-19 16:09:33,385 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id, team.id AS id_1, team.name AS name_1, team.headquarters 
FROM hero LEFT OUTER JOIN team ON team.id = hero.team_id
2023-12-19 16:09:33,386 INFO sqlalchemy.engine.Engine [generated in 0.00154s] ()
Hero: id=1 age=None name='Deadpond' secret_name='Dive Wilson' team_id=2 Team: name='Z-Force' headquarters="Sister Margaret's Bar" id=2
Hero: id=2 age=48 name='Rusty-Man' secret_name='Tommy Sharp' team_id=1 Team: name='Preventers' headquarters='Sharp Tower' id=1
Hero: id=3 age=None name='Spider-Boy' secret_name='Pedro Parqueador' team_id=None Team: None
2023-12-19 16:09:33,389 INFO sqlalchemy.engine.Engine ROLLBACK


## 连表查询

### join和where连用

In [8]:
with Session(engine) as session:
    statement = select(Hero, Team).join(Team).where(Team.name == "Preventers")
    results = session.exec(statement)
    for hero, team in results:
        print("Preventer Hero:", hero, "Team:", team)


Preventer Hero: id=2 age=48 name='Rusty-Man' secret_name='Tommy Sharp' team_id=1 Team: name='Preventers' headquarters='Sharp Tower' id=1


In [18]:
with Session(engine) as session:
    hero_no_team=session.exec(select(Hero).where(Hero.team_id==None)).first()
    print(hero_no_team)
    hero_no_team.team_id=2

    session.add(hero_no_team)
    session.commit()
    session.refresh(hero_no_team)
    print("Updated hero:", hero_no_team)

id=3 age=None name='Spider-Boy' secret_name='Pedro Parqueador' team_id=None
Updated hero: id=3 age=None name='Spider-Boy' secret_name='Pedro Parqueador' team_id=2
